In [95]:
import tensorflow as tf
import numpy as np
import ipycanvas
import ipywidgets
import igraph

In [96]:
tf.config.set_visible_devices([], 'GPU')

In [97]:
connection = np.loadtxt("connection.csv", )
pos = np.loadtxt("position.csv", )
radius = np.loadtxt("radius.csv", )

In [98]:
rest_length = np.sqrt(
    np.square((pos[:, 0:1] - (pos[:, 0:1]).transpose())) 
    + np.square((pos[:, 1:2] - (pos[:, 1:2]).transpose()))
)

In [99]:
n_particles = pos.shape[0]

connected_particles = np.unique(np.argwhere(connection - np.eye(n_particles)).flatten())
rattlers = set(range(n_particles)) - set(connected_particles)

print("Found %i rattlers" % len(rattlers))

Found 0 rattlers


In [100]:
active_layer_depth = 0.8
fixed_layer_depth = (1.0 - active_layer_depth) / 2.0
strain_unit = 1e-2 * 5
breaking_limit = 1e-2


# reverse top / bottom because canvas plots small y higher
bottom_layer = np.where(pos[:, 1] > (active_layer_depth + fixed_layer_depth), np.ones_like(pos[:, 1]), np.zeros_like(pos[:, 1]))
top_layer = np.where(pos[:, 1] < fixed_layer_depth, np.ones_like(pos[:, 1]), np.zeros_like(pos[:, 1]))
active_layer = np.ones_like(pos[:, 1]) - top_layer - bottom_layer


grad_mask = np.zeros_like(pos)
grad_mask[:, 0] = active_layer
grad_mask[:, 1] = active_layer

unit_y_strain = np.zeros_like(pos)
unit_y_strain[:, 0] = strain_unit * top_layer

In [101]:
for i in rattlers:
    radius[i] = 0.0  # removes rattlers in visualization

In [102]:
pos = tf.Variable(initial_value=pos, dtype=tf.float32)
radius = tf.constant(radius, dtype=pos.dtype)
rest_length = tf.constant(rest_length, dtype=pos.dtype)
connection = tf.constant(connection, dtype=pos.dtype)
connection_original = tf.constant(connection, dtype=pos.dtype)
breaking_limit = tf.constant(breaking_limit, dtype=pos.dtype)

unit_y_strain = tf.constant(unit_y_strain, dtype=pos.dtype)
grad_mask = tf.constant(grad_mask, dtype=pos.dtype)

half = tf.constant(0.5, dtype=pos.dtype)
one = tf.constant(1.0, dtype=pos.dtype)
zero = tf.constant(0.0, dtype=pos.dtype)
epsilon = tf.constant(1e-12, dtype=pos.dtype)

In [103]:
width = 600
padding = 200
def visual(canvas, pos, rest_length, connection, radius, broken_connection=None):
    """Plots the connection network."""
    canvas.clear()
    canvas.fill_style = "blue"
    canvas.global_alpha = 0.12 * 5
    n_particles = pos.shape[0]
    scale = width
    links = np.argwhere(connection)  # edges
    if broken_connection is not None:
        broken_links = np.argwhere(broken_connection) # broken links
    else:
        broken_links = []
    with ipycanvas.hold_canvas(canvas):
        # for i in range(n_particles):
        #     canvas.fill_circle(scale * pos[i, 0] + padding / 2, scale * pos[i, 1] + padding / 2, scale * radius[i])
        
        canvas.stroke_style = "red"
        for (i, j) in links:
            if i < j:
                extension = np.sqrt(np.square(pos[i, 0] - pos[j, 0]) + np.square(pos[i, 1] - pos[j, 1])) - (rest_length[i][j])
                canvas.line_width = scale * np.abs(extension).astype(np.float)
                canvas.stroke_line(scale * pos[i, 0] + padding / 2, scale * pos[i, 1] + padding / 2,
                                   scale * pos[j, 0] + padding / 2, scale * pos[j, 1] + padding / 2)
        
        canvas.stroke_style = "black"
        for (i, j) in links:
            if i < j:
                canvas.line_width = 1.0
                canvas.stroke_line(scale * pos[i, 0] + padding / 2, scale * pos[i, 1] + padding / 2,
                                   scale * pos[j, 0] + padding / 2, scale * pos[j, 1] + padding / 2)
        
        canvas.stroke_style = "orange"
        for (i, j) in broken_links:
            if i < j:
                extension = np.sqrt(np.square(pos[i, 0] - pos[j, 0]) + np.square(pos[i, 1] - pos[j, 1])) - (rest_length[i][j])
                canvas.line_width = 5.0
                canvas.stroke_line(scale * pos[i, 0] + padding / 2, scale * pos[i, 1] + padding / 2,
                                   scale * pos[j, 0] + padding / 2, scale * pos[j, 1] + padding / 2)
            

In [104]:
canvas = ipycanvas.Canvas(width=width+padding, height=width+padding)
visual(canvas, pos.numpy(), rest_length.numpy(), connection.numpy(), radius.numpy())
display(canvas)

Canvas(height=800, width=800)

In [105]:
@tf.function    
def energy_matrix(position: tf.Tensor, rest_length: tf.Tensor, connection: tf.Tensor):
    """Calculates the total potential energy."""
    x = position[:, 0:1]
    y = position[:, 1:2]
    dx = x - tf.transpose(x)
    dy = y - tf.transpose(y)
    distance_square_matrix = tf.square(dx) + tf.square(dy)
    cutoff_matrix = rest_length
    distance_matrix = tf.math.sqrt(epsilon + distance_square_matrix)
    energy_matrix = half * tf.math.square(tf.multiply(connection, distance_matrix - cutoff_matrix))
    energy = half * (tf.reduce_sum(energy_matrix) - tf.linalg.trace(energy_matrix))
    return energy

# @tf.function
def update_connection(position: tf.Tensor, rest_length: tf.Tensor, connection: tf.Tensor, breaking_limit: tf.Tensor):
    """Updates the connection."""
    x = position[:, 0:1]
    y = position[:, 1:2]
    dx = x - tf.transpose(x)
    dy = y - tf.transpose(y)
    distance_square_matrix = tf.square(dx) + tf.square(dy)
    cutoff_matrix = (rest_length) * (one + breaking_limit)
    distance_matrix = tf.math.sqrt(epsilon + distance_square_matrix)
    
    excess_matrix = tf.multiply(connection, distance_matrix - cutoff_matrix)
    excess_max = tf.reduce_max(excess_matrix)
    if excess_max > 0:
        connection = tf.where(excess_matrix == excess_max, tf.zeros_like(connection), connection)
    return connection

In [106]:
tf.scatter_nd(
    indices=[[1,2]], updates=[1], shape=[3,3], name=None
)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 0]], dtype=int32)>

In [107]:
energy_matrix(pos, rest_length, connection)

<tf.Tensor: shape=(), dtype=float32, numpy=1.323594e-15>

In [108]:
output = ipywidgets.widgets.Output()

In [109]:
display(canvas)

Canvas(height=800, width=800)

In [110]:
display(output)

Output()

In [111]:
energy_matrix(pos, radius, connection)

<tf.Tensor: shape=(), dtype=float32, numpy=0.30670738>

In [112]:
# schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=3e-1, decay_steps=100, decay_rate=0.95, 
# )
# optimizer = tf.keras.optimizers.SGD(learning_rate=schedule, momentum=0.9, )
schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3, decay_steps=100, decay_rate=0.9, 
)
optimizer = tf.keras.optimizers.Adam(learning_rate=schedule)


steps = 10000
strain_steps = 0
balanced_limit = 1e-6
balanced = False
for step in range(steps):
    if balanced:
        connection_new = update_connection(pos, rest_length, connection, breaking_limit)
        if np.alltrue(connection.numpy() == connection_new.numpy()):
            # new connections to break
            pos.assign_add(unit_y_strain)  # add y strain
            strain_steps += 1
        else:
            connection = connection_new  # breaks some connections
        
        balanced = False
        for var in optimizer.variables():
            var.assign(tf.zeros_like(var))   # resets optimizer memory

    with tf.GradientTape() as tape:
        total_energy = energy_matrix(pos, rest_length, connection, )
    grad = tape.gradient(total_energy, pos)
    grad = tf.multiply(grad_mask, grad)
    optimizer.apply_gradients(zip([grad], [pos]))
    with ipycanvas.hold_canvas(canvas):
        visual(canvas, pos.numpy(), rest_length.numpy(), connection.numpy(), radius.numpy(), broken_connection=(connection_original.numpy() - connection.numpy()))
    with output:
        output.clear_output()
        print("%i strain steps applied." % strain_steps)
        print("max grad = %.2e" % tf.reduce_max(tf.abs(grad)).numpy())
    if tf.reduce_max(tf.abs(grad)).numpy() < balanced_limit:   # all particles are balanced
        balanced = True

KeyboardInterrupt: 

In [ ]:
grad_mask.shape